In [1]:
import json
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-30 07:42:31.367705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 07:42:31.387138: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 07:42:31.387177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 07:42:31.400479: I tensorflow/core/platform/cpu_feature_gua

In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [3]:
with open('enriched.json') as f:
    data = json.load(f)


In [4]:
for rec in data:
    rec['search_field'] = rec['Title'].split('-')[-1] + rec['enriched_content']

In [5]:
for rec in data:
    rec['search_vector'] = model.encode(rec['search_field'])

In [6]:
data[0]

{'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
 'Text': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to",
 'Start_TimeStamp': 0.719,
 'End_TimeStamp': 30.039,
 'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s',
 'enriched_content': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want

In [8]:
data[0].keys()

dict_keys(['Title', 'Text', 'Start_TimeStamp', 'End_TimeStamp', 'link', 'enriched_content', 'search_field', 'search_vector'])

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Title": {"type": "text"},
            "Text": {"type": "text"},
            "Start_TimeStamp": {"type": "text"},
            "End_TimeStamp": {"type": "keyword"} ,
            "link": {"type": "keyword"} ,
            "enriched_content": {"type": "keyword"} ,
            "search_field": {"type": "keyword"} ,
            "search_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
        }
    }
}

In [8]:
index_name = "video-content"

es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'video-content'})

In [9]:
for doc in data:
    try:
        es.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [16]:
for doc in data:
    if '3.3' in doc['Title']:
        print(doc)

{'Title': 'llm zoomcamp 3.3.1 - evaluation metrics for retrieval', 'Text': 'hi everyone in this bunch of videos in this series of videos we want to talk about evolation evola in our search results this is the r part in our uh rck where we retrieve data from our knowledge base and there are many ways uh of storing and getting the data for the r part so we have seen min search we have seen elastic search with text only retrieval we have seen also vector search also in our cas it was elastic search and you might be wondering what', 'Start_TimeStamp': 1.48, 'End_TimeStamp': 31.119, 'link': 'https://www.youtube.com/watch?v=APMrUnC_dy0&t=1s', 'enriched_content': 'hi everyone in this bunch of videos in this series of videos we want to talk about evolation evola in our search results this is the r part in our uh rck where we retrieve data from our knowledge base and there are many ways uh of storing and getting the data for the r part so we have seen min search we have seen elastic search with

In [12]:

search_term = "3.3"
vector_search_term = model.encode(search_term)

In [14]:
query = {
    "field": "search_vector",
    "query_vector": vector_search_term,
    "k": 10,
    "num_candidates": 10000, 
}
res = es.search(index=index_name, knn=query, source=["Title","link"])
res["hits"]["hits"]

[{'_index': 'video-content',
  '_id': 'Irx3oJEBqfYKGsjjwVWP',
  '_score': 0.64927995,
  '_source': {'link': 'https://www.youtube.com/watch?v=yTO5sRw6x78&t=1339s',
   'Title': 'llm zoomcamp 4.3 - offline rag evaluation'}},
 {'_index': 'video-content',
  '_id': 'Rbx3oJEBqfYKGsjjuFMn',
  '_score': 0.64226615,
  '_source': {'link': 'https://www.youtube.com/watch?v=8KH6AS2PqWk&t=293s',
   'Title': 'llm zoomcamp 2.4 - phi 3 mini'}},
 {'_index': 'video-content',
  '_id': 'Orx3oJEBqfYKGsjjt1Pq',
  '_score': 0.63577247,
  '_source': {'link': 'https://www.youtube.com/watch?v=a86iTyxnFE4&t=882s',
   'Title': 'llm zoomcamp 2.3 - huggingface and google flan t5'}},
 {'_index': 'video-content',
  '_id': 'PLx3oJEBqfYKGsjjt1P0',
  '_score': 0.63302165,
  '_source': {'link': 'https://www.youtube.com/watch?v=8KH6AS2PqWk&t=0s',
   'Title': 'llm zoomcamp 2.4 - phi 3 mini'}},
 {'_index': 'video-content',
  '_id': 'RLx3oJEBqfYKGsjjuFMi',
  '_score': 0.63040626,
  '_source': {'link': 'https://www.youtube.com/

In [15]:
{'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to",
  'Start_TimeStamp': 0.719,
  'End_TimeStamp': 30.039,
  'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s',
  'enriched_content': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to use uh to solve throughout the course um so this will be our running problem and in our community in data do club we have multiple courses so this llm zoom camp is our fifth course and usually in our courses we have frequently asked questions so there are questions that uh there are no answers in the videos or answers are not uh easy to find and we have these documents"},
 

({'Title': 'llm zoomcamp 1.1 - introduction to llm and rag',
  'Text': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and i want to start first with explaining the problem we are going to",
  'Start_TimeStamp': 0.719,
  'End_TimeStamp': 30.039,
  'link': 'https://www.youtube.com/watch?v=Q75JgLEXMsM&t=0s',
  'enriched_content': "hi everyone welcome to our course this is our first module for first unit so in this course the course is called llm zoom camp in this course we will learn about practical applications of llm and in particular we will focus our attention on rack retrieval a generation i'll shortly talk about these variations what they mean um and what we exactly will do and 